In [1]:
# 라이브러리 로딩

from pandas_ods_reader import read_ods
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import argparse
import getpass
import json
import pandas as pd
import pprint
import sys
import time

In [2]:
wait_time_out = 10
driver = webdriver.Chrome()
wait = W(driver, wait_time_out)
driver.maximize_window()

### Login to Autoway

In [3]:
# 로그인
def login(driver, userid, passwd, otp):
            
    # 카테고리 (서비스) 선택
    driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]/div[3]/div/div[5]/label/div[1]/i').click()
    
    # 회사 선택
    driver.find_element_by_xpath('//*[@id="drpGroupCopList"]').click()

    value = "현대오토에버"
    select = Select(driver.find_element_by_id('drpGroupCopList'))
    select.select_by_visible_text(value)

    # UserID 입력
    username = driver.find_element_by_id("txtLoginID_Visible")
    username.clear()
    username.send_keys(userid)

    # Password 입력
    password = driver.find_element_by_id("password")
    password.clear()
    password.send_keys(passwd)

    # OTP 입력
    username = driver.find_element_by_id("code")
    username.clear()
    username.send_keys(otp)

    # Login 버튼 클릭
    login = driver.find_element_by_xpath('//*[@id="Login"]').click()

    return login

# ifrMain 으로 이동
def gobacktoMain(driver):
    driver.switch_to.default_content()
    iframe = driver.find_element_by_xpath('//iframe[@id="ifrMain"]')
    driver.switch_to.frame(iframe)

# ifrGnbMenu iframe 으로 이동
def clickTools(driver):
    driver.find_element_by_xpath('//*[@id="btnGnbMenu"]/i').click()
    time.sleep(0.5)
    iframe = driver.find_element_by_xpath('//iframe[@id="ifrGnbMenu"]')
    driver.switch_to.frame(iframe)
    driver.find_element_by_xpath('//*[@id="gnb_perfect_scroll"]/div[3]/a').click()
    time.sleep(0.5)
    
# i-sams 문자열로 app 찾기    
def searchiSams(driver):
    # 검색어 입력
    search = driver.find_element_by_id("ctl00_ContentPlaceHolder1_txtSearchText")
    search.clear()
    search.send_keys('i-sams')
    search.send_keys(Keys.ENTER)
    time.sleep(0.5)
    
# i-sams 팝업창 띄우기
def clickLinkofiSams(driver):
    ret = driver.find_element_by_xpath('//*[@id="aspnetForm"]/div[5]/div/div[2]/div[2]/div/div[2]/div[2]/table/tbody/tr[2]/td[2]/a[1]').click()
    print(ret)
    time.sleep(0.5)

# i-sams 팝업창에서 링크 찾아 이동하기
def gotoPopUp_and_movetoiSams(driver):
    iframe = driver.find_element_by_xpath('//iframe[@name="InfoPopUpFrame"]')
    driver.switch_to.frame(iframe)
    # html = driver.page_source
    # html
    ret = driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div/div[1]/div[2]/table/tbody/tr[6]/td/div/a').click()
    print(ret)
    time.sleep(0.5)
    
# 진단실행 하기
def execute(driver, index, hostname):
    try:               
        # 진단 실행 대상 선택 (체크박스)
        time.sleep(3)
        checkboxes = driver.find_elements_by_class_name('asset-td-checkbox')
        total_rows_length = len(checkboxes)

        if total_rows_length > 0:
            for i in range(0, total_rows_length):
                checkboxes[i].click()
            time.sleep(1)
            
            try:
                # 진단 실행 버튼 Click
                execute = driver.find_element_by_id("bt-auditstart")
                execute.click()
                time.sleep(1)

                try:
                    # 진단 실행 Popup 창 Accept 확인
                    execute = driver.find_element_by_id("bt-modal-startaudit-ok")
                    execute.click()
                    time.sleep(1)
                finally:
                    # 진단 실행 Popup 창 이후 경고 창 확인
                    alert_obj = driver.switch_to.alert
                    # print(alert_obj)
                    alert_obj.accept()
                    # print(driver.window_handles)
                    print("Test requested : ", index, hostname)
                    time.sleep(1)
            except:
                pass
        else:
            print("Hostname doesn't exists : ", index, hostname)
    finally:
        # 자산 탭 버튼 Click
        execute = driver.find_element_by_id("tree-tab-asset")
        execute.click()
        time.sleep(1)
        
# 진단실행 취소 하기
def cancelExecute(driver, index, hostname):

    try:
        # 진단 실행 Popup 창 취소 버튼
        execute = driver.find_element_by_xpath('//*[@id="modal-startaudit"]/div[1]/table/tbody[2]/tr/td/div/button[2]')
        execute.click()
        print("Test cancelled : ", index, hostname)
        time.sleep(1)

    finally:
        # 자산 탭 버튼 Click
        execute = driver.find_element_by_id("tree-tab-asset")
        execute.click()
        time.sleep(1)

In [4]:
# 카테고리 (서비스) 선택
while(1):
    try:
        driver.get('https://autoway.hyundai.net/login')
        element = W(driver, 10).until(E.presenence_of_element_located((By.ID, "drpGroupCopList")))
    finally:
        break

In [5]:
userid = '9372821'
passwd = 'Rlawldbs0713**'
otp = '20828084'

ret = login(driver, userid, passwd, otp)
time.sleep(5)
print(ret)

None


### Login to i-Sams

In [6]:
print(driver.current_window_handle)

print(driver.window_handles)

driver.switch_to.window(driver.window_handles[0])

# 기본 iframe 으로 이동
driver.switch_to.default_content()

# ifrGnbMenu iframe 으로 이동후, 도구 메뉴 클릭
clickTools(driver)

# ifrMain (IT Tools 화면 내) 으로 이동
gobacktoMain(driver)

# i-sams 문자열로 app 찾기  
searchiSams(driver)

# i-sams 팝업창 띄우기
clickLinkofiSams(driver)

# i-sams 팝업창에서 링크 찾아 이동하기
gotoPopUp_and_movetoiSams(driver)
time.sleep(15)

print(driver.current_window_handle)

# html = driver.page_source
# html

print(driver.window_handles)

driver.switch_to.window(driver.window_handles[1])

print(driver.current_window_handle)

CDwindow-1853883B0051E7E9F6698F05B013FB1C
['CDwindow-1853883B0051E7E9F6698F05B013FB1C']
None
None
CDwindow-1853883B0051E7E9F6698F05B013FB1C
['CDwindow-1853883B0051E7E9F6698F05B013FB1C', 'CDwindow-0AC6758A4B16892AE9DA15E032FB8B6C']
CDwindow-0AC6758A4B16892AE9DA15E032FB8B6C


### Data Loading

In [7]:
base_path = r"./21년도 정보시스템 및 인프라 정기점검_11월02일기준_모니터링(전체).ods"
sheet = 'Sheet'

df = read_ods(base_path, sheet, headers=True)

# df.head()
# df.count()

df = df[(df["정담당자 부서명"] == "클라우드운영센터") & (df["점검상태.1"] == "미점검")]

# df.count()
# df.describe()
# df.head()

df = df[['HOSTNAME', '담당팀명', '담당자명', '점검상태.1']]

# df.head()

df = df.set_axis(["hostname", "team", "owner", "status"], axis=1)

# df.head()
# df.describe()

df.reset_index(inplace=True)

# df.head()

print(driver.window_handles)
print(driver.current_window_handle)

['CDwindow-1853883B0051E7E9F6698F05B013FB1C', 'CDwindow-0AC6758A4B16892AE9DA15E032FB8B6C']
CDwindow-0AC6758A4B16892AE9DA15E032FB8B6C


# Collection

In [8]:
def HostAvailability(driver, hostname):
    try:
        search = driver.find_element_by_id("input-text-list-asset-search")
        time.sleep(3)
        search.clear()
        search.click()
        search.send_keys(hostname)
        search.send_keys(Keys.ENTER)
        time.sleep(3)

        try:            
            element_present = W(driver, 10).until(EC.presence_of_element_located((By.ID, "assetlink-hint-text")))            
            if element_present.text == '데이터가 없습니다':
                ret = False
            else:
                ret = True
        except TimeoutException:
            print ("Timed out waiting for page to load")
            pass
    except:
        pass
    return ret

In [9]:
def checkHostAvailability(driver, row):
    
    ret = False
    
    try:
        hostname = row['hostname'].split(".")[0]+"."+row['hostname'].split(".")[1]
    except:
        hostname = row['hostname'].split(".")[0]
        
    try:
        ret = HostAvailability(driver, hostname)
        if ret == False:
            hostname = row['hostname'].split(".")[0]
            ret = HostAvailability(driver, hostname)
        else:
            pass
    except:   
        print(hostname, "is not available")
    finally:
        return ret, hostname

In [10]:
def gotoReport(driver):
    # 리포트 탭 버튼 Click
    reportTab = driver.find_element_by_id("tab-report-menu")
    reportTab.click()
    time.sleep(1)

    # 리포트 Dropdown 버튼 Click
    report = driver.find_element_by_xpath('//*[@id="report-list-0"]/td[5]/table/tbody/tr[1]/td[7]/div[3]/button')
    report.click()
    time.sleep(1)

In [1]:
def collectReport(driver, hostname):
    df_ret = pd.DataFrame()
    
    try:
        gotoReport(driver)
        
        items = driver.find_elements_by_class_name('bt-report-item-view-inner.ellipsis')
#         print(len(items)) # Task 숫자를 (화면에서) 보고 싶을 때

        num_failed = 0
        try:
            items_failed = driver.find_elements_by_class_name('ss-bt-position-list.one-circle-text.report-item-select-fail-list')
            num_failed = len(items_failed)
        except:
            num_failed = 0
    
        count = 0
        tasks = []
        results = []
        
        for idx, e in enumerate(items):
            tasks.append(e.get_attribute("title"))

            if idx < num_failed:
                results.append('failed')
            else:
                results.append('success')

        df_ret = pd.DataFrame(results, index = tasks, columns = [ hostname ])
        
    except:
        pass
    
    finally:
        return df_ret

In [2]:
def main(df):
    count = 0
    availability = False
    timeout = 5

    # 작업 시작 위치
    start_index = 0
    end_index = 10000 # 최대 만개까지

    for index, row in df.iterrows():
        idx = int(index)
        if idx >= start_index and idx < end_index:
            availability, hostname = checkHostAvailability(driver, row)
            if availability == True:
                df_ret = collectReport(driver, hostname)
                result.append(df_ret)
                print("HA", index, hostname)
            else:
                print("HNA", index, hostname)
                continue
#             break  # 1개만 TEST 로 돌리고 싶을 때
        else:
            pass
    return result

In [3]:
result = []
    
if __name__ == "__main__":
    result = main(df)

NameError: name 'df' is not defined

In [31]:
print(driver.window_handles)
print(driver.current_window_handle)

['CDwindow-17FA7B198DC6912CA9AE95AED8ABB4ED', 'CDwindow-A626CFBB1C1F8C2219FA053EAE8F1C57']
CDwindow-A626CFBB1C1F8C2219FA053EAE8F1C57


In [14]:
driver.quit()

# Saving Report Collections

In [82]:
df_result = pd.DataFrame()

for df in result:
    if df_result.empty:
        df_result = df.reset_index()
    else:
        df_temp = df.reset_index()
        df_result = pd.merge(df_result, df_temp, on='index', how='outer')

In [86]:
df_result.set_index('index', inplace=True)

In [92]:
df_T = df_result.T

In [93]:
df_T.to_csv("tasks_04.csv", index=True)

In [65]:
# from pyexcel_ods import save_data
# save_data("tasks.ods", df_result)

In [32]:
print("총 갱신 레코드 / 요청 작업 수 / 갱신 성공(작업 수): ========== %d / %d / %d" % (total, checkRequest, count) )

총 갱신 레코드 / 요청 작업 수 / 갱신 성공(작업 수): ========== 383 / 0 / 0


In [22]:
# html = driver.page_source
# html

In [ ]:
# 이슈 있는 호스트
# 83 ~ 86

In [ ]:
# %load isams.py
# (env) D:\rpa>python isams.py --ods_fn '2021년 i-sams 조치 대상 리스트_0902.ods' --owner 장재봉 --id 9306532 --pwd Tmvkzm1!